In [23]:
import pandas as pd
import os
import arff

# Wczytanie danych

In [24]:
fp = 'input/254516_online.csv'
data = pd.read_csv(fp)
data.head()

,status pozyczki,kwota wnioskowana,kwota otrzymana,PESEL klienta,rodzaj źródła dochodu,miesieczny dochod netto,ktore rolowanie,data decyzji,data początkowa dochodu,data koncowa dochodu,planowany termin splaty,rzeczywisty termin spłaty
0,splacona_cz,800.0,800.0,490702xxx4x,13,3700.00,3,2011-08-31 16:26:31,2009-07-13,0000-00-00,2011-09-30,2011-09-30 00:00:00
1,splacona_cz,300.0,300.0,480124xxx2x,13,1100.00,0,2011-08-31 17:37:10,2003-01-01,0000-00-00,2011-09-30,2011-09-30 00:00:00
2,splacona_cz,400.0,400.0,460902xxx5x,16,1473.04,0,2011-09-01 09:15:50,1998-01-01,0000-00-00,2011-10-01,2011-09-30 00:00:00
3,splacona_cz,400.0,350.0,330729xxx3x,13,1519.00,0,2011-09-01 09:32:40,1990-08-16,0000-00-00,2011-10-01,2011-10-01 00:00:00
4,splacona_cz,200.0,200.0,741203xxx8x,12,599.03,2,2011-09-01 09:51:44,2010-03-01,2011-09-30,2011-10-01,2011-10-03 00:00:00


# Ćwiczenie 1

In [25]:
data['kwota kredytu'] = data.apply(lambda row: row['kwota wnioskowana'] if row['status pozyczki'] == 'odmowa' else row['kwota otrzymana'], axis=1)
data[['status pozyczki', 'kwota wnioskowana', 'kwota otrzymana', 'kwota kredytu']].head(10)

,status pozyczki,kwota wnioskowana,kwota otrzymana,kwota kredytu
0,splacona_cz,800.00,800.00,800.00
1,splacona_cz,300.00,300.00,300.00
2,splacona_cz,400.00,400.00,400.00
3,splacona_cz,400.00,350.00,350.00
4,splacona_cz,200.00,200.00,200.00
5,splacona_cz,200.00,200.00,200.00
6,splacona_cz,200.00,200.00,200.00
7,splacona_cz,218.32,218.32,218.32
8,splacona_cz,393.99,393.99,393.99
9,odmowa,1000.00,0.00,1000.00


# Ćwiczenie 2

In [26]:
def calculate_birth_date(pesel):
    year, month, day = int(pesel[:2]), int(pesel[2:4]), pesel[4:6]
    if month > 12:
        year += 2000
        month -= 20
    else:
        year += 1900
    month = f"{month:02d}"
    return f"{year}-{month}-{day}"


test_pesel = "012820xxx5x"
print(calculate_birth_date(test_pesel))

data['data urodzenia'] = data['PESEL klienta'].apply(calculate_birth_date)
data[['PESEL klienta', 'data urodzenia']].head()

2001-08-20


,PESEL klienta,data urodzenia
0,490702xxx4x,1949-07-02
1,480124xxx2x,1948-01-24
2,460902xxx5x,1946-09-02
3,330729xxx3x,1933-07-29
4,741203xxx8x,1974-12-03


# Ćwiczenie 3

In [27]:
data['data decyzji'] = pd.to_datetime(data['data decyzji'])
data['data urodzenia'] = pd.to_datetime(data['data urodzenia'])

data['wiek'] = data['data decyzji'].dt.year - data['data urodzenia'].dt.year
data[['data decyzji', 'data urodzenia', 'wiek']].head()

,data decyzji,data urodzenia,wiek
0,2011-08-31 16:26:31,1949-07-02,62
1,2011-08-31 17:37:10,1948-01-24,63
2,2011-09-01 09:15:50,1946-09-02,65
3,2011-09-01 09:32:40,1933-07-29,78
4,2011-09-01 09:51:44,1974-12-03,37


# Ćwiczenie 4

In [28]:
data['płeć'] = data['PESEL klienta'].apply(lambda x: 'kobieta' if int(x[9]) % 2 == 0 else 'mężczyzna')
data[['PESEL klienta', 'płeć']].head()

,PESEL klienta,płeć
0,490702xxx4x,kobieta
1,480124xxx2x,kobieta
2,460902xxx5x,mężczyzna
3,330729xxx3x,mężczyzna
4,741203xxx8x,kobieta


# Ćwiczenie 5

In [29]:
data['data początkowa dochodu'] = pd.to_datetime(data['data początkowa dochodu'])
data['okres w jakim pobierał dochód'] = data.apply(lambda row:
                                                   (row['data decyzji'].year - row['data początkowa dochodu'].year) * 12 +
                                                   (row['data decyzji'].month - row['data początkowa dochodu'].month), axis=1)

data[['data decyzji', 'data początkowa dochodu', 'okres w jakim pobierał dochód']].head()

,data decyzji,data początkowa dochodu,okres w jakim pobierał dochód
0,2011-08-31 16:26:31,2009-07-13,25
1,2011-08-31 17:37:10,2003-01-01,103
2,2011-09-01 09:15:50,1998-01-01,164
3,2011-09-01 09:32:40,1990-08-16,253
4,2011-09-01 09:51:44,2010-03-01,18


# Ćwiczenie 6

In [30]:
data['data początkowa dochodu dt'] = pd.to_datetime(data['data początkowa dochodu'], format='%Y-%m-%d', errors='coerce')
data['data koncowa dochodu dt'] = pd.to_datetime(data['data koncowa dochodu'], format='%Y-%m-%d', errors='coerce')
data['data decyzji dt'] = pd.to_datetime(data['data decyzji'], format='%Y-%m-%d %H:%M:%S', errors='coerce')


def full_months_between(d1, d2):
    if d1 is pd.NaT or d2 is pd.NaT:
        return None
    year_diff = d2.year - d1.year
    month_diff = d2.month - d1.month
    return year_diff * 12 + month_diff


data['okres w jakim będzie pobierał dochód'] = data.apply(
    lambda row: 48 if pd.isna(row['data koncowa dochodu dt'])
    else full_months_between(row['data początkowa dochodu dt'], row['data koncowa dochodu dt']),
    axis=1
)

data['okres w jakim będzie pobierał dochód'] = data['okres w jakim będzie pobierał dochód'].clip(upper=48)

data[['data początkowa dochodu', 'data koncowa dochodu', 'data decyzji', 'okres w jakim będzie pobierał dochód']].head()

,data początkowa dochodu,data koncowa dochodu,data decyzji,okres w jakim będzie pobierał dochód
0,2009-07-13,0000-00-00,2011-08-31 16:26:31,48
1,2003-01-01,0000-00-00,2011-08-31 17:37:10,48
2,1998-01-01,0000-00-00,2011-09-01 09:15:50,48
3,1990-08-16,0000-00-00,2011-09-01 09:32:40,48
4,2010-03-01,2011-09-30,2011-09-01 09:51:44,18


# Ćwiczenie 7

In [31]:
data['planowany termin spłaty dt'] = pd.to_datetime(data['planowany termin splaty'], format='%Y-%m-%d', errors='coerce')
data['rzeczywisty termin spłaty dt'] = pd.to_datetime(data['rzeczywisty termin spłaty'], errors='coerce')

dzisiejsza_data = pd.to_datetime('today').normalize()

data['opóźnienie spłaty'] = data.apply(lambda row: (row['rzeczywisty termin spłaty dt'] - row['planowany termin spłaty dt']).days
if not pd.isna(row['planowany termin spłaty dt']) and not pd.isna(row['rzeczywisty termin spłaty dt'])
else (dzisiejsza_data - row['planowany termin spłaty dt']).days
if pd.isna(row['rzeczywisty termin spłaty dt']) and not pd.isna(row['planowany termin spłaty dt'])
else 0, axis=1)

data['opóźnienie spłaty'] = data['opóźnienie spłaty'].clip(lower=0)

data[['planowany termin splaty', 'rzeczywisty termin spłaty', 'opóźnienie spłaty']].head(100)

,planowany termin splaty,rzeczywisty termin spłaty,opóźnienie spłaty
0,2011-09-30,2011-09-30 00:00:00,0
1,2011-09-30,2011-09-30 00:00:00,0
2,2011-10-01,2011-09-30 00:00:00,0
3,2011-10-01,2011-10-01 00:00:00,0
4,2011-10-01,2011-10-03 00:00:00,2
...,...,...,...
95,2011-10-10,2011-10-10 00:00:00,0
96,2011-10-10,2011-10-11 00:00:00,1
97,2011-10-10,2011-10-10 00:00:00,0
98,2011-10-10,2011-10-14 00:00:00,4


# Ćwiczenie 8

In [34]:
columns_mapping = {
    'status pozyczki': 'status pożyczki',
    'miesieczny dochod netto': 'miesięczny dochód netto',
    'ktore rolowanie': 'które rolowanie',
}
data.rename(columns=columns_mapping, inplace=True)

selected_columns = [
    'status pożyczki',
    'kwota kredytu',
    'wiek',
    'płeć',
    'rodzaj źródła dochodu',
    'miesięczny dochód netto',
    'które rolowanie',
    'okres w jakim pobierał dochód',
    'okres w jakim będzie pobierał dochód',
    'opóźnienie spłaty'
]

output_folder_path = 'output'

if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

output_file_path = 'output/output.csv'
# output_file_path = 'output/output.xlsx'

data_to_export = data[selected_columns]

# data_to_export.to_excel(output_file_path, index=False)
data_to_export.to_csv(output_file_path, index=False, encoding='utf-8')

In [33]:
# data = pd.read_excel(output_file_path)
data_csv = pd.read_csv(output_file_path)

arff_data = {
    'attributes': [(column_name, 'NUMERIC') if data[column_name].dtype.kind in 'ifc' else (column_name, 'STRING')
                   for column_name in data.columns],
    'data': data.values,
    'relation': 'relation_name',
    'description': ''
}

with open('output/output_weka.arff', 'w', encoding='utf-8') as f:
    arff.dump(arff_data, f)

TypeError: expected str, bytes or os.PathLike object, not dict

In [41]:
import csv
fileToRead = 'output/output.csv'  #csv file name or absolute path to be open.
fileToWrite = "output/weka.arff"  #name as how you'll save your arff file.
relation = "ZATD"  #how you'll like to call your relation as.

dataType = []  # Stores data types 'nominal' and 'numeric'
columnsTemp = []  # Temporary stores each column of csv file except the attributes
uniqueTemp = []  # Temporary Stores each data cell unique of each column
uniqueOfColumn = []  # Stores each data cell unique of each column
dataTypeTemp = []  # Temporary stores the data type for cells on each column
finalDataType = []  # Finally stores data types 'nominal' and 'numeric'
attTypes = []  # Stores data type 'numeric' and nominal data for attributes
p = 0  # pointer for each cell of csv file

writeFile = open(fileToWrite, 'w', encoding='utf-8')

#Opening and Reading a CSV file
f = open(fileToRead, 'r', encoding='utf-8')
reader = csv.reader(f)
allData = list(reader)
attributes = allData[0]
totalCols = len(attributes)
totalRows = len(allData)
f.close()

# Add a '0' for each empty cell
for j in range(0, totalCols):
    for i in range(0, totalRows):
        if 0 == len(allData[i][j]):
            allData[i][j] = "0"

# check for comams or blanks and adds single quotes
for j in range(0, totalCols):
    for i in range(1, totalRows):
        allData[i][j] = allData[i][j].lower()
        if "\r" in allData[i][j] or '\r' in allData[i][j] or "\n" in allData[i][j] or '\n' in allData[i][j]:
            allData[i][j] = allData[i][j].rstrip(os.linesep)
            allData[i][j] = allData[i][j].rstrip("\n")
            allData[i][j] = allData[i][j].rstrip("\r")
        try:
            if allData[i][j] == str(float(allData[i][j])) or allData[i][j] == str(int(allData[i][j])):
                print
        except ValueError as e:
            allData[i][j] = "'" + allData[i][j] + "'"

# fin gives unique cells for nominal and numeric
for j in range(0, totalCols):
    for i in range(1, totalRows):
        columnsTemp.append(allData[i][j])
    for item in columnsTemp:
        if not (item in uniqueTemp):
            uniqueTemp.append(item)
    uniqueOfColumn.append("{" + ','.join(uniqueTemp) + "}")
    uniqueTemp = []
    columnsTemp = []

# Assigns numeric or nominal to each cell
for j in range(1, totalRows):
    for i in range(0, totalCols):
        try:
            if allData[j][i] == str(float(allData[j][i])) or allData[j][i] == str(int(allData[j][i])):
                dataType.append("numeric")
        except ValueError as e:
            dataType.append("nominal")

for j in range(0, totalCols):
    p = j
    for i in range(0, (totalRows - 1)):
        dataTypeTemp.append(dataType[p])
        p += totalCols
    if "nominal" in dataTypeTemp:
        finalDataType.append("nominal")
    else:
        finalDataType.append("numeric")
    dataTypeTemp = []

for i in range(0, len(finalDataType)):
    if finalDataType[i] == "nominal":
        attTypes.append(uniqueOfColumn[i])
    else:
        attTypes.append(finalDataType[i])

# Show comments
writeFile.write("%\n% Comments go after a '%' sign.\n%\n")
writeFile.write("%\n% Relation: " + relation + "\n%\n%\n")
writeFile.write("% Attributes: " + str(totalCols) + " " * 5
                + "Instances: " + str(totalRows - 1) + "\n%\n%\n\n")

# Show Relation
writeFile.write("@relation " + relation + "\n\n")

# Show Attributes
for i in range(0, totalCols):
    writeFile.write("@attribute" + " '" + attributes[i]
                    + "' " + attTypes[i] + "\n")

# Show Data
writeFile.write("\n@data\n")
for i in range(1, totalRows):
    writeFile.write(','.join(allData[i]) + "\n")

print(fileToWrite + " was converted to " + fileToRead)

output/weka.arff was converted to output/output.csv
